# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using AtomsBuilder
using DFTK
using LinearAlgebra
using PseudoPotentialData

pseudopotentials = PseudoFamily("dojo.nc.sr.pbesol.v0_4_1.oncvpsp3.standard.upf")
model = model_DFT(bulk(:Si); functionals=PBEsol(), pseudopotentials)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.397835393820                   -0.90    5.2   26.3ms
  2   -8.400233952942       -2.62       -1.74    1.0   41.1ms
  3   -8.400405678669       -3.77       -2.97    1.5   20.0ms
  4   -8.400428711835       -4.64       -2.96    3.5   25.3ms
  5   -8.400428787220       -7.12       -3.01    1.0   18.8ms
  6   -8.400429017826       -6.64       -4.63    1.0   18.9ms
  7   -8.400429023927       -8.21       -4.42    2.2   38.6ms
  8   -8.400429024354       -9.37       -4.86    1.0   19.4ms
  9   -8.400429024417      -10.20       -6.31    1.0   19.2ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -8.397828904980                   -0.90           5.5    469ms
  2   -8.400386908637       -2.59       -1.79   0.80    2.0    546ms
  3   -8.400423419600       -4.44       -3.05   0.80    1.0    174ms
  4   -8.400428985456       -5.25       -3.41   0.80    2.5   21.0ms
  5   -8.400429021810       -7.44       -4.55   0.80    1.2   16.6ms
  6   -8.400429024385       -8.59       -5.69   0.80    2.2   19.9ms
  7   -8.400429024420      -10.45       -6.35   0.80    3.2   22.3ms


## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   +0.795511903617                   -1.04    2.83s
  2   -1.771030668195        0.41       -0.67    111ms
  3   -3.713049252149        0.29       -0.39   39.2ms
  4   -4.706567883807       -0.00       -0.45   66.1ms
  5   -6.444802619533        0.24       -0.53   39.0ms
  6   -7.426841501136       -0.01       -0.72   38.9ms
  7   -7.594715307324       -0.78       -1.28   28.9ms
  8   -8.120954287068       -0.28       -1.39   47.1ms
  9   -8.177483465506       -1.25       -1.62   29.0ms
 10   -8.280635168351       -0.99       -1.42   39.0ms
 11   -8.319235595240       -1.41       -1.59   49.7ms
 12   -8.352367537235       -1.48       -2.02   33.2ms
 13   -8.370923500937       -1.73       -2.52   28.8ms
 14   -8.382256485294       -1.95       -2.51   28.8ms
 15   -8.390075777938       -2.11       -2.60   32.1ms
 16   -8.394105133244       -2.39       -3.11   29.1ms
 17   -8.39

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.397816659995                   -0.90    5.2   26.6ms


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -8.400428843177                   -1.78    25.2s
  2   -8.400429024420       -6.74       -4.02    2.28s
  3   -8.400429024420      -14.45       -7.80    126ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 1.1795282597072996e-6
|ρ_newton - ρ_scfv| = 1.430851510398848e-7
|ρ_newton - ρ_dm|   = 3.1170757848405248e-6
